<a href="https://colab.research.google.com/github/hikaruyaku/for_work/blob/master/Membrane_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 膜分離モジュールのシミュレーター

## 作成の経緯
Excelのマクロで組んだらクソofクソの遅さなので，python で作成する．

In [19]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

## 使い方
下の記入フォームを記載した後，`Ctrl + F9`を押す．

In [31]:
#@title 記入フォーム
#@markdown 前提条件を記載してください．

#@markdown フィード流量=kg/h, 初期温度=℃, 操作圧力 = kPaA


膜の単位数 = 1000  #@param {type: "slider", min: 10, max: 1000}
膜本数 = 2000  #@param {type: "number"}
フィード流量 = 15000  #@param {type: "number"}
初期温度 = 130  #@param {type: "number"}
操作圧力_kPaA = 10  #@param {type: "number"}

#@markdown ---


## 前提条件

In [21]:
# 物性
## Antoine 定数
A_h2o = 7.07
B_h2o = 1657.46
C_h2o = 227.02
A_acoh = 6.42452
B_acoh = 1479.02
C_acoh = 216.81
## wilson の活量係数
Lambda_11 = 3351.51
Lambda_12 = 29.6876
## ガス透過係数
Q_h2o = 5e-6
Q_acoh = 1.25e-8
## 膜厚
sigma = 1e-5
## 分子量
mw_h2o = 18
mw_acoh = 60
## 顕熱[kJ/mol*K]
Cp_h2o = 0.0756
Cp_acoh = 0.1292
## 潜熱[kJ/mol]
h_h2o = 40.7
h_acoh = 24.4
## 活量係数
gamma_h2o = 0.00389
gamma_acog = 0.00387

In [20]:
# 膜特性
ID = 0.012
L = 0.7
N = 膜本数
length = L/膜の単位数

# 膜の面積も出しちゃう
dA = 0.012*math.pi*length

## 関数

In [32]:
# Antoineでの蒸気圧の出す関数

def antoine_funk(a,b,c,T):
  p_ast = 10 ** (a-b/(c+T))
  return p_ast

# dL1 を出す関数
def dl_1(Temp, x, yp):
  dl_1 = Q_h2o / sigma * ( gamma_h2o * antoine_funk(A_h2o,B_h2o,C_h2o,Temp) * x - 操作圧力_kPaA * yp ) * dA
  return dl_1
# dL2 を出す関数
def dl_2(Temp, x, yp):
  dl_2 = Q_acoh / sigma * ( gamma_h2o * antoine_funk(A_acoh,B_acoh,C_acoh,Temp)*(1-x) - 操作圧力_kPaA * (1-yp) ) * dA
  return dl_2

# 蒸気透過後の温度を出す関数
def temp_cal(l_h2o,l_acoh,v_h2o,v_acoh):
  delta_temp = (v_h2o * h_h2o + v_acoh * h_acoh)/1000/(l_h2o * Cp_h2o + l_acoh * Cp_acoh)
  return delta_temp

## 本計算
- 本計算は `for`文を用いた繰り返し計算を行う．
- また `for` 文内で，マテリアルバランスの式と膜透過の式の連立方程式を解く．

$$
\begin{eqnarray}
\frac{dL_1}{dA} &=& -\frac{Q_1}{\sigma}(\gamma_1 p_1^*x - p_1y_p) \\
\frac{dL_2}{dA} &=& -\frac{Q_2}{\sigma}(\gamma_2 p_2^*(1-x) - p_1(1-y_p)) \\
(dL_1 + dL_2) * y_p &=& dL_1
\end{eqnarray}
$$